In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchtext sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 23.3 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompat

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import ViTFeatureExtractor, ViTModel
from sentence_transformers import SentenceTransformer
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import nltk
from nltk.translate.bleu_score import sentence_bleu
import os
import nltk
from nltk.translate.bleu_score import sentence_bleu

import pickle
from tqdm import tqdm
import h5py

In [ ]:
EMBED_SIZE = 768  # ViT has 12 layers
HIDDEN_SIZE = 512
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.001

    # Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
def extract_vit_features(image, feature_extractor, vit_model):
    """Extract features from ViT"""
    with torch.no_grad():
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = vit_model(**inputs)
        features = outputs.last_hidden_state[:, 0, :]
        print(f"ViT feature shape: {features.shape}")
        return features

In [ ]:
class DataPreprocessor:
    def __init__(self, image_dir, captions_file, feature_extractor, max_len=50, cache_dir='cached_data'):
        self.image_dir = image_dir
        self.captions_file = captions_file  # Added this line
        self.max_len = max_len
        self.cache_dir = cache_dir
        self.feature_extractor = feature_extractor
        self.vit_model = None

        # Create cache directory
        os.makedirs(cache_dir, exist_ok=True)

        # Cache file paths
        self.vocab_cache = os.path.join(cache_dir, 'vocabulary.pkl')
        self.train_cache = os.path.join(cache_dir, 'train_data.pkl')
        self.test_cache = os.path.join(cache_dir, 'test_data.pkl')

        # Initialize ViT model (do it once)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        print("Loading ViT model...")
        self.vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(self.device)
        self.vit_model.eval()

    def extract_features(self, image_path):
        try:
            image = Image.open(image_path).convert('RGB')

            with torch.no_grad():
                inputs = self.feature_extractor(images=image, return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                outputs = self.vit_model(**inputs)
                features = outputs.last_hidden_state[:, 0, :].cpu()
                return features.squeeze(0)
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
            return torch.zeros(768)

    def process_data(self):
        if (os.path.exists(self.train_cache) and
            os.path.exists(self.test_cache) and
            os.path.exists(self.vocab_cache)):
            print("Loading cached data...")
            return self.load_cached_data()

        print("Processing data from scratch...")
        return self.create_and_cache_data()

    def create_and_cache_data(self):
        print("Reading captions file...")
        img_captions = {}
        all_captions = []

        # Read and process captions
        with open(self.captions_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()[1:]  # Skip header
            for line in tqdm(lines, desc="Reading captions"):
                parts = line.strip().split(',', 1)
                if len(parts) == 2:
                    img_name = parts[0].strip()
                    caption = parts[1].strip().strip('"\'')

                    if caption:
                        if img_name not in img_captions:
                            img_captions[img_name] = []
                        img_captions[img_name].append(caption)
                        all_captions.append(caption)

        # Build vocabulary
        word2idx, idx2word = self.build_vocabulary(all_captions)

        # Process all images and captions
        print("\nProcessing images and creating batches...")
        features_list = []
        captions_list = []

        for img_name in tqdm(img_captions.keys(), desc="Processing images"):
            image_path = os.path.join(self.image_dir, img_name)
            if os.path.exists(image_path):
                features = self.extract_features(image_path)

                for caption in img_captions[img_name]:
                    features_list.append(features)

                    # Process caption
                    words = caption.lower().split()
                    caption_indices = [word2idx.get(word, word2idx['<UNK>']) for word in words]
                    caption_indices = [word2idx['<START>']] + caption_indices + [word2idx['<END>']]

                    # Pad sequence
                    if len(caption_indices) < self.max_len:
                        caption_indices += [word2idx['<PAD>']] * (self.max_len - len(caption_indices))
                    else:
                        caption_indices = caption_indices[:self.max_len]

                    captions_list.append(caption_indices)
            else:
                print(f"Warning: Image not found: {image_path}")

        # Convert to tensors
        print("\nConverting to tensors...")
        features_tensor = torch.stack(features_list)
        captions_tensor = torch.tensor(captions_list)

        # Split into train and test
        print("Splitting into train and test sets...")
        indices = torch.randperm(len(features_tensor))
        train_size = int(0.8 * len(indices))

        train_indices = indices[:train_size]
        test_indices = indices[train_size:]

        train_data = (features_tensor[train_indices], captions_tensor[train_indices])
        test_data = (features_tensor[test_indices], captions_tensor[test_indices])

        # Cache the processed data
        print("Caching processed data...")
        with open(self.train_cache, 'wb') as f:
            pickle.dump(train_data, f)
        with open(self.test_cache, 'wb') as f:
            pickle.dump(test_data, f)

        print(f"\nProcessing completed!")
        print(f"Train set size: {len(train_indices)}")
        print(f"Test set size: {len(test_indices)}")

        return (word2idx, idx2word), train_data, test_data
    def build_vocabulary(self, captions):
        print("Building vocabulary...")
        word_freq = {}

        for caption in captions:
            words = caption.lower().split()
            for word in words:
                word_freq[word] = word_freq.get(word, 0) + 1

        word2idx = {'<PAD>': 0, '<START>': 1, '<END>': 2, '<UNK>': 3}
        for word, freq in sorted(word_freq.items(), key=lambda x: x[1], reverse=True):
            if len(word2idx) < 10000:
                word2idx[word] = len(word2idx)

        idx2word = {v: k for k, v in word2idx.items()}

        # Cache vocabulary
        with open(self.vocab_cache, 'wb') as f:
            pickle.dump((word2idx, idx2word), f)

        print(f"Vocabulary size: {len(word2idx)}")
        return word2idx, idx2word
    def load_cached_data(self):
        print("Loading vocabulary...")
        with open(self.vocab_cache, 'rb') as f:
            vocab = pickle.load(f)

        print("Loading train data...")
        with open(self.train_cache, 'rb') as f:
            train_data = pickle.load(f)

        print("Loading test data...")
        with open(self.test_cache, 'rb') as f:
            test_data = pickle.load(f)

        print(f"Train set size: {len(train_data[0])}")
        print(f"Test set size: {len(test_data[0])}")

        return vocab, train_data, test_data

In [ ]:

from torch.utils.data import Dataset, DataLoader, TensorDataset

    # Initialize ViT feature extractor
print("Initializing ViT feature extractor...")
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

    # Create dataset
preprocessor = DataPreprocessor(
        image_dir="/content/drive/MyDrive/Tech India/Preprocessed-Dataset/Rams-approach-preprocess/flickr30k/Images",
        captions_file="/content/drive/MyDrive/Tech India/Preprocessed-Dataset/Rams-approach-preprocess/flickr30k/captions.txt",
        feature_extractor=feature_extractor
    )
(word2idx, idx2word), (train_features, train_captions), (test_features, test_captions) = preprocessor.process_data()
# print(f"Dataset size: {len(dataset)}")
    # Split dataset
train_dataset = TensorDataset(train_features, train_captions)
test_dataset = TensorDataset(test_features, test_captions)
    # Create data loaders
train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=16,
        pin_memory=True
    )

test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=16,
        pin_memory=True
    )

Initializing ViT feature extractor...
Using device: cuda
Loading ViT model...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processing data from scratch...
Reading captions file...


Reading captions: 100%|██████████| 158915/158915 [00:00<00:00, 751378.19it/s]


Building vocabulary...
Vocabulary size: 10000

Processing images and creating batches...


Processing images:   3%|▎         | 918/31783 [03:53<2:11:00,  3.93it/s]


KeyboardInterrupt: 

In [ ]:
(word2idx, idx2word), (train_features, train_captions), (test_features, test_captions) = preprocessor.process_data()
# print(f"Dataset size: {len(dataset)}")


Processing data from scratch...
Reading captions file...


Reading captions: 100%|██████████| 158915/158915 [00:00<00:00, 844690.69it/s]


Building vocabulary...
Vocabulary size: 10000

Processing images and creating batches...


Processing images:   4%|▍         | 1339/31783 [04:29<1:42:10,  4.97it/s]


KeyboardInterrupt: 

In [ ]:
import pickle

def load_dataset(cache_dir='cached_data', batch_size=32):
    """
    Load preprocessed data from pickle files and create DataLoaders
    Returns vocabulary and data loaders for train and test sets
    """
    vocab_path = os.path.join(cache_dir, 'vocabulary.pkl')
    train_path = os.path.join(cache_dir, 'train_data.pkl')
    test_path = os.path.join(cache_dir, 'test_data.pkl')

    # Check if pickle files exist
    if not all(os.path.exists(p) for p in [vocab_path, train_path, test_path]):
        raise FileNotFoundError("Required pickle files not found. Run preprocessing first.")

    # Load vocabulary
    print("Loading vocabulary...")
    with open(vocab_path, 'rb') as f:
        word2idx, idx2word = pickle.load(f)

    # Load train features and captions
    print("Loading train data...")
    with open(train_path, 'rb') as f:
        train_features, train_captions = pickle.load(f)

    # Load test features and captions
    print("Loading test data...")
    with open(test_path, 'rb') as f:
        test_features, test_captions = pickle.load(f)

    # Create datasets
    train_dataset = TensorDataset(train_features, train_captions)
    test_dataset = TensorDataset(test_features, test_captions)

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=16,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=16,
        pin_memory=True
    )

    print(f"Vocabulary size: {len(word2idx)}")
    print(f"Train set size: {len(train_features)}")
    print(f"Test set size: {len(test_features)}")

    # Return both raw data and loaders
    raw_data = {
        'vocab': (word2idx, idx2word),
        'train_data': (train_features, train_captions),
        'test_data': (test_features, test_captions)
    }

    loaders = {
        'train': train_loader,
        'test': test_loader
    }

    return raw_data, loaders
# Load both raw data and DataLoaders
raw_data, loaders = load_dataset(cache_dir='/content/drive/MyDrive/Tech India/Preprocessed-Dataset/Rams-approach-preprocess/flickr30k/cached_data', batch_size=32)

# Access vocabulary and raw data if needed
word2idx, idx2word = raw_data['vocab']
train_features, train_captions = raw_data['train_data']
test_features, test_captions = raw_data['test_data']

# Access DataLoaders
train_loader = loaders['train']
test_loader = loaders['test']

Loading vocabulary...
Loading train data...
Loading test data...
Vocabulary size: 10000
Train set size: 127131
Test set size: 31783


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Model

In [ ]:
import torch.nn.functional as F
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.v.data.normal_(mean=0, std=0.1)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: (batch_size, 1, hidden_size)
        encoder_outputs: (batch_size, seq_len, hidden_size)
        """
        batch_size, seq_len, hidden_size = encoder_outputs.size()

        # Ensure hidden has correct shape
        if hidden.dim() == 2:
            hidden = hidden.unsqueeze(1)

        # Repeat hidden state for each encoder output
        hidden = hidden.repeat(1, seq_len, 1)

        # Calculate attention scores
        energy = torch.tanh(self.attention(torch.cat((hidden, encoder_outputs), dim=2)))

        # Reshape v for batch processing
        v = self.v.repeat(batch_size, 1).unsqueeze(1)

        # Calculate attention weights
        attention_weights = torch.bmm(v, energy.transpose(1, 2)).squeeze(1)
        attention_weights = F.softmax(attention_weights, dim=1)

        # Apply attention to encoder outputs
        context = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)

        return context, attention_weights


In [ ]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size, embed_size=256, num_layers=2, dropout_p=0.3):
        super(ImageCaptioningModel, self).__init__()

        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.num_layers = num_layers
        self.embed_size = embed_size

        # Image feature processing
        self.feature_encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_p)
        )

        # Word embeddings
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.embed_dropout = nn.Dropout(dropout_p)

        # Additional embedding processing
        self.embed_process = nn.Linear(embed_size, hidden_size)

        # Attention
        self.attention = AttentionLayer(hidden_size)

        # Decoder LSTM
        self.decoder_rnn = nn.LSTM(
            input_size=hidden_size * 2,  # Concatenated context and processed embedding
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_p if num_layers > 1 else 0
        )

        # Output projection
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden_size, vocab_size)
        )

        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self, images, captions, teacher_forcing_ratio=0.5):
        batch_size = images.size(0)
        max_length = captions.size(1) - 1  # -1 because we don't predict for last token
        device = images.device

        # Encode images
        image_features = self.feature_encoder(images)
        image_features = image_features.unsqueeze(1)  # (batch_size, 1, hidden_size)

        # Initialize outputs tensor
        outputs = torch.zeros(batch_size, max_length, self.vocab_size).to(device)

        # Initialize decoder input
        decoder_input = captions[:, 0]  # Start tokens

        # Initialize hidden states
        h = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        hidden = (h, c)

        for t in range(max_length):
            # Embed input tokens
            embedded = self.embedding(decoder_input)  # (batch_size, embed_size)
            embedded = self.embed_dropout(embedded)
            embedded = self.embed_process(embedded)  # (batch_size, hidden_size)

            # Add sequence dimension
            embedded = embedded.unsqueeze(1)  # (batch_size, 1, hidden_size)

            # Calculate attention
            context, _ = self.attention(embedded, image_features)

            # Combine embedding and context
            decoder_input_combined = torch.cat((embedded, context), dim=2)

            # RNN forward pass
            output, hidden = self.decoder_rnn(decoder_input_combined, hidden)

            # Process output
            output = self.layer_norm(output.squeeze(1))
            output = self.output_layer(output)

            # Store output
            outputs[:, t] = output

            # Teacher forcing
            if random.random() < teacher_forcing_ratio and t < max_length - 1:
                decoder_input = captions[:, t + 1]
            else:
                decoder_input = output.argmax(dim=1)

        return outputs

In [ ]:

def evaluate_model(model, test_loader, criterion, device, pad_idx):
    model.eval()
    total_loss = 0
    total_word_accuracy = 0
    total_sentence_accuracy = 0
    num_batches = 0

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating")

        for images, captions in progress_bar:
            # Move to device
            images = images.to(device)
            captions = captions.to(device)

            # Get input and target sequences
            input_captions = captions[:, :-1]
            target_captions = captions[:, 1:]

            # Forward pass
            outputs = model(images, input_captions)
            outputs = outputs[:, :-1, :]

            # Reshape for loss calculation
            outputs_flat = outputs.reshape(-1, outputs.size(-1))
            targets_flat = target_captions.reshape(-1)

            # Calculate metrics
            loss = criterion(outputs_flat, targets_flat)
            word_acc, sent_acc = calculate_accuracy(outputs_flat, targets_flat, pad_idx)

            # Update metrics
            total_loss += loss.item()
            total_word_accuracy += word_acc
            total_sentence_accuracy += sent_acc
            num_batches += 1

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'word_acc': f'{word_acc:.4f}',
                'sent_acc': f'{sent_acc:.4f}'
            })

    # Calculate averages
    avg_loss = total_loss / num_batches
    avg_word_acc = total_word_accuracy / num_batches
    avg_sent_acc = total_sentence_accuracy / num_batches

    return {
        'loss': avg_loss,
        'word_accuracy': avg_word_acc,
        'sentence_accuracy': avg_sent_acc
    }

In [ ]:
def calculate_accuracy(outputs, targets, pad_idx):
    """
    Calculate word-level
    outputs: (batch_size * seq_len, vocab_size)
    targets: (batch_size * seq_len)
    """
    # Get predictions
    predictions = outputs.argmax(dim=1)  # (batch_size * seq_len)

    # Create mask to ignore padding tokens
    mask = (targets != pad_idx)

    # Word-level accuracy
    correct_words = ((predictions == targets) & mask).sum().item()
    total_words = mask.sum().item()
    word_accuracy = correct_words / total_words if total_words > 0 else 0

    # Reshape for sentence-level accuracy
    batch_size = len(targets) // targets.shape[0]
    predictions = predictions.view(-1, batch_size)
    targets = targets.view(-1, batch_size)
    mask = mask.view(-1, batch_size)

    return word_accuracy

In [ ]:
from torch.nn.parallel import DataParallel
from tqdm import tqdm
import threading
from queue import Queue
import random
from torch.cuda.amp import autocast, GradScaler

In [ ]:
import random
def train_model(model, train_loader, criterion, optimizer, device, epoch, total_epochs, teacher_forcing_ratio=0.5):
    model.train()
    total_loss = 0
    total_words = 0
    correct_words = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{total_epochs}")

    for i, (images, captions) in enumerate(progress_bar):
        try:
            # Move to device
            images = images.to(device)
            captions = captions.to(device)

            # Forward pass
            outputs = model(images, captions)

            # Calculate loss
            loss = criterion(
                outputs.reshape(-1, outputs.size(-1)),
                captions[:, 1:].reshape(-1)
            )

            # Calculate accuracy
            predictions = outputs.argmax(dim=2)
            mask = captions[:, 1:] != 0  # Ignore padding
            correct = (predictions == captions[:, 1:]) & mask
            total_words += mask.sum().item()
            correct_words += correct.sum().item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            # Update metrics
            total_loss += loss.item()
            current_accuracy = correct_words / total_words if total_words > 0 else 0

            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_accuracy:.4f}'
            })

            # Print batch statistics
            if (i + 1) % 100 == 0:
                print(f"\nBatch {i+1}/{len(train_loader)}")
                print(f"Loss: {loss.item():.4f}")

        except Exception as e:
            print(f"\nError in batch {i}:")
            print(f"Exception: {str(e)}")
            continue

    avg_loss = total_loss / len(train_loader)
    avg_accuracy = correct_words / total_words if total_words > 0 else 0

    return avg_loss, avg_accuracy



In [ ]:
import random
import torch
import threading
from queue import Queue
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def train_model(model, train_loader, criterion, optimizer, device, epoch, total_epochs, teacher_forcing_ratio=0.5, num_threads=2):
    model.train()
    total_loss = 0
    total_words = 0
    correct_words = 0

    # Create queues for batch processing and results
    batch_queue = Queue(maxsize=num_threads * 2)
    result_queue = Queue()

    # Lock for synchronizing updates
    update_lock = threading.Lock()

    def process_batch(batch_data):
        try:
            images, captions = batch_data
            batch_results = {}

            # Move to device
            images = images.to(device)
            captions = captions.to(device)

            # Forward pass
            outputs = model(images, captions)

            # Calculate loss
            loss = criterion(
                outputs.reshape(-1, outputs.size(-1)),
                captions[:, 1:].reshape(-1)
            )

            # Calculate accuracy
            predictions = outputs.argmax(dim=2)
            mask = captions[:, 1:] != 0  # Ignore padding
            correct = (predictions == captions[:, 1:]) & mask

            batch_results['loss'] = loss
            batch_results['correct'] = correct.sum().item()
            batch_results['total'] = mask.sum().item()

            return batch_results

        except Exception as e:
            print(f"\nError processing batch:")
            print(f"Exception: {str(e)}")
            return None

    def update_metrics(results):
        nonlocal total_loss, total_words, correct_words

        with update_lock:
            if results:
                total_loss += results['loss'].item()
                total_words += results['total']
                correct_words += results['correct']

                # Backward pass (needs to be done in main thread for thread safety)
                optimizer.zero_grad()
                results['loss'].backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{total_epochs}")

    # Create thread pool
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []

        for i, batch in enumerate(progress_bar):
            # Submit batch processing to thread pool
            future = executor.submit(process_batch, batch)
            futures.append(future)

            # Process completed batches
            while futures:
                done_futures = [f for f in futures if f.done()]
                for future in done_futures:
                    results = future.result()
                    update_metrics(results)
                    futures.remove(future)

                    # Update progress bar
                    current_accuracy = correct_words / total_words if total_words > 0 else 0
                    progress_bar.set_postfix({
                        'loss': f'{total_loss/(i+1):.4f}',
                        'acc': f'{current_accuracy:.4f}'
                    })

                    # Print batch statistics
                    if (i + 1) % 100 == 0:
                        print(f"\nBatch {i+1}/{len(train_loader)}")
                        print(f"Loss: {total_loss/(i+1):.4f}")
                        print(f"Accuracy: {current_accuracy:.4f}")
                        print(f"Total words: {total_words}")
                        print(f"Correct words: {correct_words}")

        # Process any remaining futures
        for future in futures:
            results = future.result()
            update_metrics(results)

    avg_loss = total_loss / len(train_loader)
    avg_accuracy = correct_words / total_words if total_words > 0 else 0

    return avg_loss, avg_accuracy


'\nmodel = YourModel().to(device)\ncriterion = torch.nn.CrossEntropyLoss(ignore_index=0)\noptimizer = torch.optim.Adam(model.parameters(), lr=0.001)\n\nfor epoch in range(total_epochs):\n    avg_loss, avg_accuracy = train_model(\n        model=model,\n        train_loader=train_loader,\n        criterion=criterion,\n        optimizer=optimizer,\n        device=device,\n        epoch=epoch+1,\n        total_epochs=total_epochs,\n        num_threads=2  # Adjust based on your system\n    )\n'

In [ ]:
def main():
    LEARNING_RATE = 0.001
    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    model = ImageCaptioningModel(
        input_size=768,  # ViT base size
        hidden_size=HIDDEN_SIZE,
        vocab_size=len(word2idx),
        embed_size=EMBED_SIZE,
        num_layers=2,
        dropout_p=0.3
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding index
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # Training loop
    print("Starting training...")
    for epoch in range(EPOCHS):
        loss, accuracy = train_model(
        model=model,
        train_loader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epoch=epoch+1,
        total_epochs=EPOCHS,
        num_threads=16 # Adjust based on your system
    )

        print(f"\nEpoch {epoch + 1}/{EPOCHS}")
        print(f"Average Loss: {loss:.4f}")

        # Save checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'accuracy': accuracy,
            'word2idx': word2idx,
            'idx2word': idx2word
        }, f'improved_model_epoch_{epoch+1}.pth')

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print(f"Main error: {str(e)}")
        raise

Using device: cuda
Starting training...


Epoch 1/10:   3%|▎         | 100/3973 [01:22<36:32,  1.77it/s, loss=5.4345, acc=0.1859]


Batch 100/3973
Loss: 5.4345
Accuracy: 0.1859
Total words: 46286
Correct words: 8605


Epoch 1/10:   5%|▌         | 200/3973 [02:40<2:18:31,  2.20s/it, loss=5.1775, acc=0.2087]


Batch 200/3973
Loss: 5.1775
Accuracy: 0.2087
Total words: 92146
Correct words: 19230


...

Epoch 1/10:  91%|█████████ | 3600/3973 [46:37<03:12,  1.94it/s, loss=4.5450, acc=0.2607]


Batch 3600/3973
Loss: 4.5450
Accuracy: 0.2607
Total words: 1657903
Correct words: 432262


Epoch 1/10:  93%|█████████▎| 3700/3973 [47:55<02:25,  1.88it/s, loss=4.5389, acc=0.2611]


Batch 3700/3973
Loss: 4.5389
Accuracy: 0.2611
Total words: 1703370
Correct words: 444669


Epoch 1/10:  93%|█████████▎| 3703/3973 [48:02<09:10,  2.04s/it, loss=4.5388, acc=0.2610]

In [ ]:
def generate_caption(image_path, model_path="best_model.pth", max_length=50):
    """
    Generate a caption for a single image using the saved model
    """
    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location='cpu')
    word2idx = checkpoint['word2idx']
    idx2word = checkpoint['idx2word']

    # Initialize model and load weights
    model = ImageCaptioningModel(
        input_size=768,  # ViT base size
        hidden_size=512,
        vocab_size=len(word2idx)
    )
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Load and process image
    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
    vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')

    # Extract features
    image = Image.open(image_path).convert('RGB')

    with torch.no_grad():
        # Get ViT features
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = vit_model(**inputs)
        image_features = outputs.last_hidden_state[:, 0, :]  # [1, 768]
        print(f"Image Embedding shape:",{image_features.shape})
    # Generate caption
    with torch.no_grad():
        # Initialize with start token
        current_token = torch.tensor([[word2idx['<START>']]])
        caption = []

        # Generate words until max length or end token
        for _ in range(max_length):
            # Generate next word
            output = model(image_features, current_token)
            next_word_idx = output[0, -1].argmax().item()

            # Convert to word
            word = idx2word[next_word_idx]

            # Stop if end token or pad
            if word in ['<END>', '<PAD>']:
                break

            caption.append(word)

            # Update current token
            current_token = torch.cat([current_token, torch.tensor([[next_word_idx]])], dim=1)

    return ' '.join(caption)

# Example usage:
if __name__ == "__main__":
    # Test with a sample image
    image_path = "/content/8192398089.jpg"  # Replace with your image path
    caption = generate_caption(image_path)
    print(f"\nGenerated caption: {caption}")

    # # Test with multiple images
    # test_images = [
    #     "flickr30k/images/image1.jpg",
    #     "flickr30k/images/image2.jpg",
    #     "flickr30k/images/image3.jpg"
    # ]

    # print("\nGenerating captions for multiple images:")
    # for img_path in test_images:
    #     try:
    #         caption = generate_caption(img_path)
    #         print(f"\nImage: {img_path}")
    #         print(f"Caption: {caption}")
    #     except Exception as e:
    #         print(f"Error processing {img_path}: {str(e)}")

<ipython-input-28-38f54335be68>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')
/usr/local/lib/python3.10/dist-packa

Image Embedding shape: {torch.Size([1, 768])}

Generated caption: women black in and outfits in dance


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
def load_model_and_tokenizers(model_path):
    """Load the trained model and tokenizers"""
    checkpoint = torch.load(model_path, map_location='cpu')
    word2idx = checkpoint['word2idx']
    idx2word = checkpoint['idx2word']

    # Initialize model
    model = ImageCaptioningModel(
        input_size=768,  # ViT base size
        hidden_size=512,
        vocab_size=len(word2idx)
    )

    model.load_state_dict(checkpoint['model_state_dict'])
    return model, word2idx, idx2word

def generate_caption(model, image_features, word2idx, idx2word, max_length=50):
    """Generate a caption for given image features"""
    model.eval()
    with torch.no_grad():
        # Initialize with start token
        decoder_input = torch.tensor([[word2idx['<START>']]])

        # Initialize hidden states
        h = torch.zeros(model.num_layers, 1, model.hidden_size)
        c = torch.zeros(model.num_layers, 1, model.hidden_size)
        hidden = (h, c)

        caption = []

        for _ in range(max_length):
            # Forward pass through model
            output, hidden = model(image_features.unsqueeze(0), decoder_input, hidden)

            # Get predicted word
            pred_idx = output[0, -1].argmax().item()
            pred_word = idx2word[pred_idx]

            # Break if end token or padding
            if pred_word in ['<END>', '<PAD>']:
                break

            caption.append(pred_word)
            decoder_input = torch.tensor([[pred_idx]])

    return caption

def calculate_bleu_scores(references, hypothesis):
    """Calculate BLEU-1,2,3,4 scores"""
    smoothing = SmoothingFunction().method1

    # Ensure references is a list of lists
    if not isinstance(references[0], list):
        references = [references]

    # Calculate individual BLEU scores
    bleu1 = sentence_bleu(references, hypothesis, weights=(1, 0, 0, 0), smoothing_function=smoothing)
    bleu2 = sentence_bleu(references, hypothesis, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
    bleu3 = sentence_bleu(references, hypothesis, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing)
    bleu4 = sentence_bleu(references, hypothesis, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)

    return {
        'bleu1': bleu1,
        'bleu2': bleu2,
        'bleu3': bleu3,
        'bleu4': bleu4
    }

def evaluate_model(model_path, test_dataloader, device):
    """Evaluate model on test set"""
    # Load model and tokenizers
    model, word2idx, idx2word = load_model_and_tokenizers(model_path)
    model = model.to(device)
    model.eval()

    # Initialize feature extractor
    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
    vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)

    # Initialize metrics
    all_bleu_scores = []
    all_references = []
    all_hypotheses = []

    print("Generating captions and calculating BLEU scores...")
    for images, captions in tqdm(test_dataloader):
        try:
            images = images.to(device)

            # Generate captions
            generated_caption = generate_caption(model, images, word2idx, idx2word)

            # Get reference captions (remove special tokens)
            reference_captions = []
            for cap in captions:
                tokens = [idx2word[idx.item()] for idx in cap]
                # Remove special tokens
                tokens = [token for token in tokens if token not in ['<START>', '<END>', '<PAD>', '<UNK>']]
                reference_captions.append(tokens)

            # Calculate BLEU scores
            scores = calculate_bleu_scores(reference_captions, generated_caption)
            all_bleu_scores.append(scores)

            # Store for corpus BLEU calculation
            all_references.append(reference_captions)
            all_hypotheses.append(generated_caption)

        except Exception as e:
            print(f"Error processing batch: {str(e)}")
            continue

    # Calculate average scores
    avg_scores = {
        'bleu1': np.mean([s['bleu1'] for s in all_bleu_scores]),
        'bleu2': np.mean([s['bleu2'] for s in all_bleu_scores]),
        'bleu3': np.mean([s['bleu3'] for s in all_bleu_scores]),
        'bleu4': np.mean([s['bleu4'] for s in all_bleu_scores])
    }

    # Calculate corpus BLEU
    corpus_bleu1 = corpus_bleu(all_references, all_hypotheses, weights=(1, 0, 0, 0))
    corpus_bleu4 = corpus_bleu(all_references, all_hypotheses, weights=(0.25, 0.25, 0.25, 0.25))

    # Print results
    print("\nEvaluation Results:")
    print("\nAverage BLEU Scores:")
    print(f"BLEU-1: {avg_scores['bleu1']:.4f}")
    print(f"BLEU-2: {avg_scores['bleu2']:.4f}")
    print(f"BLEU-3: {avg_scores['bleu3']:.4f}")
    print(f"BLEU-4: {avg_scores['bleu4']:.4f}")

    print("\nCorpus BLEU Scores:")
    print(f"Corpus BLEU-1: {corpus_bleu1:.4f}")
    print(f"Corpus BLEU-4: {corpus_bleu4:.4f}")

    # Save some example predictions
    print("\nSample Predictions:")
    for i in range(min(5, len(all_hypotheses))):
        print(f"\nImage {i+1}:")
        print(f"Generated: {' '.join(all_hypotheses[i])}")
        print(f"Reference: {' '.join(all_references[i][0])}")

    return avg_scores, {'corpus_bleu1': corpus_bleu1, 'corpus_bleu4': corpus_bleu4}

def evaluate_single_image(model, image_path, word2idx, idx2word, feature_extractor, vit_model, device):
    """Generate caption for a single image"""
    model.eval()
    with torch.no_grad():
        # Load and process image
        image = Image.open(image_path).convert('RGB')
        inputs = feature_extractor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Extract features
        features = vit_model(**inputs).last_hidden_state[:, 0, :]

        # Generate caption
        caption = generate_caption(model, features, word2idx, idx2word)

        return ' '.join(caption)

if __name__ == "__main__":
    # Parameters
    MODEL_PATH = 'best_model.pth'  # Path to your saved model
    BATCH_SIZE = 32
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load test dataset
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4
    )

    # Evaluate model
    avg_scores, corpus_scores = evaluate_model(MODEL_PATH, test_loader, device)

    # Test on single images
    model, word2idx, idx2word = load_model_and_tokenizers(MODEL_PATH)
    model = model.to(device)
    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
    vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)

    # Test on some sample images
    test_images = [
        "/content/8192398089.jpg",
    ]

    print("\nTesting individual images:")
    for img_path in test_images:
        try:
            caption = evaluate_single_image(
                model, img_path, word2idx, idx2word,
                feature_extractor, vit_model, device
            )
            print(f"\nImage: {img_path}")
            print(f"Generated Caption: {caption}")
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-44-5302034d90fe>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary o

NameError: name 'EncoderDecoder' is not defined